# Step 2 - Snapping

In this notebook, we take our origin points (here, derived from a population raster manually using QGIS's 'Raster pixels to Points' tool, and saved as as shapefile. 

At the end of this notebook, we are ready to generate the OD Matrix

In [1]:
import os, sys
import time
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point, MultiPoint
from shapely.wkt import loads
from scipy import spatial
from functools import partial
import pyproj
from shapely.ops import transform

import GOSTnets as gn
import geopy

from datetime import datetime

Set path names and file names, and import Graph

In [2]:
simplif_meters = 25

In [3]:
source_epsg = 4326
target_epsg = 3106

In [4]:
input_pth = r'P:\BGD\GEO'
interm_pth = r'intermediate'
fin_pth = r'final'
res_pth = r'results'

In [5]:
# scenario = 'current'
scenario = 'Padma'

In [6]:
# WorldPop data parameters
# Assumes pre-processing of WP data into points dataset using ArcGIS/QGIS to project, resample, and convert to points

constraint_status = 'constrained'
# constraint_status = 'unconstrained'

wp_res = 100
# wp_res = 250
# wp_rs = '1k'

In [7]:
# Production date for outputs being created

# Production date for outputs being used

prod_date = '210329'
# prod_date = '210503'
# prod_date = '210518'
# prod_date = '201524'
print(prod_date)

210329


In [8]:
# make prod_date subfolders if they don't already exist

if not os.path.exists(os.path.join(fin_pth,prod_date)):
    os.makedirs(os.path.join(fin_pth,prod_date))

# make results prod_date folder

if not os.path.exists(os.path.join(res_pth,prod_date)):
    os.makedirs(os.path.join(res_pth,prod_date))
    os.makedirs(os.path.join(res_pth,prod_date,'spatial'))
    os.makedirs(os.path.join(res_pth,prod_date,'tables'))

Load data

In [18]:
fin_pckle = f'final_{scenario}_G_{simplif_meters}m_{prod_date}.pickle'

In [25]:
G = nx.read_gpickle(os.path.join(fin_pth, fin_pckle))

In [9]:
# Load in origins
print(constraint_status, wp_res)
origin_file = f'bgd_wp_{constraint_status}_origins_{wp_res}m_2020.shp' 
print(origin_file)

constrained 100
bgd_wp_constrained_origins_100m_2020.shp


In [ ]:
# Destinations

city_fil = r'Population\\Places\\bd_major_cities_pop_projs.gpkg'
dhaka_chitt_fil = r'Population\\Places\\bd_cities_DhakaChittOnly_pop_projs.gpkg'
minor_city_fil = r'Population\\Places\\bd_cities_noDhakaChitt_pop_projs.gpkg'
dry_ports_fil = r'Economic\\Ports\\bd_landports_active.gpkg'
deep_sea_ports_fil = r'Economic\\Ports\\bd_DeepSeaPorts.gpkg'
river_ports_fil = r'P:\BGD\GEO\Team\Infrastructure_Diagnostic\Nodes\river_ports.gpkg'

sezs_all_fil = r'Economic\SEZs\SEZs_all_pt_4326.gpkg'
sezs_active_fil = r'Economic\SEZs\SEZs_active_pt_4326.gpkg'


In [ ]:
dests = {"All_cities" : city_fil, "Dhaka_Chitt" : dhaka_chitt_fil, "Minor_cities" : minor_city_fil, \
         "Dry_ports" : dry_ports_fil, "River_ports" : river_ports_fil, "Deep_sea_ports" : deep_sea_ports_fil, \
         'All_SEZs' : sezs_all_fil, 'Active_SEZs' : sezs_active_fil}

# dests = {"All_cities" : city_fil, "Dhaka_Chitt" : dhaka_chitt_fil, "Minor_cities" : minor_city_fil, \
#          "Dry_ports" : dry_ports_fil, "River_ports" : river_ports_fil, "Deep_sea_ports" : deep_sea_ports_fil}

# dests = {'All_SEZs' : sezs_all_fil, 'Active_SEZs' : sezs_active_fil}

### Origins

Import Origins

In [10]:
origins = gpd.read_file(os.path.join(input_pth,f'Population\WorldPop\{constraint_status}',origin_file))

In [10]:
len(origins)

2199542

In [14]:
origins.dtypes

VALUE        float64
geometry    geometry
dtype: object

In [15]:
origins.head(10)

,VALUE,geometry
0,75.282043,POINT (342002.622 2946479.439)
1,93.414307,POINT (342251.347 2946479.439)
2,65.594666,POINT (342500.072 2946479.439)
3,46.771957,POINT (341505.172 2946230.714)
4,95.297745,POINT (341753.897 2946230.714)
5,110.515930,POINT (342002.622 2946230.714)
6,107.968536,POINT (342251.347 2946230.714)
7,116.065910,POINT (342500.072 2946230.714)
8,19.164389,POINT (341007.722 2945981.989)
9,101.732681,POINT (341256.447 2945981.989)


Optional: convert MultiPoint to Point and clip origins _exactly_ to project extent

In [26]:
# Optional cleaning step -- convert MultiPoint to Point (necessary for pandana snap to work)

if origins.loc[0].geometry.geom_type == 'MultiPoint':
    
    origins = gpd.GeoDataFrame(
        origins, geometry=gpd.points_from_xy(origins.centroid.x,origins.centroid.y)) # manually re-calculate as a point geometry
else:
    None

In [18]:
# Optional filter (clip) origins by enclosing object

# shp = gpd.read_file(os.path.join('inputs/cxb_filters/', 'cxb_dist_4326.shp'))
# shp = shp.to_crs({'epsg:4326'})
# cxb_obj = shp.geometry.iloc[0]

# cxb_pop_filter = origins.within(cxb_obj)
# origins = origins.loc[cxb_pop_filter]
# origins.crs = "EPSG:4326"
# len(origins) # CXB polygon clipped

Snap Origins

In [19]:
origins_snapped = gn.pandana_snap_c(G, 
                                      origins, 
                                      source_crs='epsg:{}'.format(source_epsg),
                                      target_crs='epsg:{}'.format(target_epsg), 
                                      add_dist_to_node_col = True,
                                      time_it=True)

time elapsed for function
18.15324592590332


In [20]:
origins_snapped.head()

,VALUE,geometry,NN,NN_dist
0,75.282043,POINT (342002.622 2946479.439),53985,111.746558
1,93.414307,POINT (342251.347 2946479.439),53985,138.744438
2,65.594666,POINT (342500.072 2946479.439),267861,176.743207
3,46.771957,POINT (341505.172 2946230.714),88169,234.058416
4,95.297745,POINT (341753.897 2946230.714),111725,240.743236


In [21]:
# max.origins_snapped.VALUE

In [22]:
origins_snapped.rename({'bgdppp2020U' : 'VALUE'},axis=1,inplace=True)

Save Origins to origin folder

In [23]:
len(origins_snapped)

2216419

In [24]:
origins_snapped.to_csv(os.path.join(res_pth, prod_date, origin_file.replace('.shp',f'_snapped_{simplif_meters}m.csv')))

### Destinations

First let's take a look at the destinations dict

In [13]:
print(dests)

{'All_cities': 'Population\\\\Places\\\\bd_major_cities_pop_projs.gpkg', 'Dhaka_Chitt': 'Population\\\\Places\\\\bd_cities_DhakaChittOnly_pop_projs.gpkg', 'Minor_cities': 'Population\\\\Places\\\\bd_cities_noDhakaChitt_pop_projs.gpkg', 'Dry_ports': 'Economic\\\\Ports\\\\bd_landports_active.gpkg', 'River_ports': 'P:\\BGD\\GEO\\Team\\Infrastructure_Diagnostic\\Nodes\\river_ports.gpkg', 'Deep_sea_ports': 'Economic\\\\Ports\\\\bd_DeepSeaPorts.gpkg'}


Hiding annoying error messages related to projection formatting

In [14]:
import warnings
warnings.filterwarnings('ignore')

Read in, snap, and export snapped destinations from the *dests* dictionary

In [15]:
for dest, fpth in dests.items():
    
    print(dest)
    
    # Read in the file, convert MultiPoint to Point
    
    gdf = gpd.read_file(os.path.join(input_pth,fpth))
    gdf.crs = "EPSG:4326"
#     gdf.to_crs({'init':'epsg:4326'})
    gdf = gpd.GeoDataFrame(
        gdf, geometry=gpd.points_from_xy(gdf.centroid.x, gdf.centroid.y),crs="EPSG:4326") # geometries must be manually converted to Point, not MultiPoint
    
    # Pandana snap everything
    
    gdf_snapped = gn.pandana_snap_c(G, 
                                     gdf, 
                                     source_crs='epsg:{}'.format(source_epsg),
                                     target_crs='epsg:{}'.format(target_epsg), 
                                     add_dist_to_node_col = True,
                                     time_it=True)
    
    # Export the finished snapped destination to a CSV format for processing in Step 3

    export_name = dest + f'_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv'
    gdf_snapped.to_csv(os.path.join(fin_pth, prod_date, export_name))

All_cities
time elapsed for function
7.869895935058594
Dhaka_Chitt
time elapsed for function
7.98689603805542
Minor_cities
time elapsed for function
7.133907318115234
Dry_ports
time elapsed for function
8.644887685775757
River_ports
time elapsed for function
6.776906490325928
Deep_sea_ports
time elapsed for function
7.605892896652222
